In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
    Column,
    NumeralTickFormatter,
    Label
    )
from bokeh.io import show, output_notebook
from bokeh.models.widgets import Panel, Tabs
from bokeh.plotting import figure , output_file, show
from subprocess import check_output

print(check_output(["ls", "../input"]).decode("utf8"))

#reading the dataset
data1 = pd.read_csv("../input/Sunspots.csv",sep = ",", index_col = False)
data1 = data1[data1.columns[1:]]
data1y = list(data1["Monthly Mean Total Sunspot Number"])

output_notebook()

def proc1(data,yr) :
     
    dataC = data["Date"]
    
    for n in range(0, len(data)) :
        tmp = dataC[n][0:4]
        yr.append(int(tmp))
        
    return (yr)

year = []

proc1(data1,year)

year1 = []
for n in year :
    if n not in year1 :
        year1.append(n)
        
# Preparing the dataframe

mat_sun = pd.DataFrame(columns = ("Date","Year","Month","Spots","MA","Data"))

m = ["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]

year    = []
month   = []
spots   = []


for n in range(0,len(data1)) :
    ano = int(str(data1["Date"][n])[0:4])
    mes = int(str(data1["Date"][n])[5:7])
    year.append(ano)
    month.append(m[mes])
    spots.append(round(data1y[n],3))

ma = data1["Monthly Mean Total Sunspot Number"].rolling(window = 33).mean()


#a = list(pd.to_datetime(data1["Date"], format = "%Y-%m-%d"))
#print (a)
#print (type(a))

mat_sun["Date"]     = list(pd.to_datetime(data1["Date"], format = "%Y-%m-%d"))
mat_sun["Year"]     = year
mat_sun["Month"]    = month
mat_sun["Spots"]    = spots
mat_sun["MA"]       = list(ma)
dd = []
for n in data1["Date"] :
    dd.append(str(n)[0:10]) 
mat_sun["Data"]    = dd

#Heatmap
def heatmap(data, colors, title, year1, months) : # hacked from Bokeh´s gallery

  mapper = LinearColorMapper(palette=colors, low=data.Spots.min(), high=data.Spots.max())
  source = ColumnDataSource(data)

  years =  list(year1)
  months = list(months)

  years = map(str,years)
  months = map(str,months)
  
  
  TOOLS = "hover, pan,box_zoom,reset,wheel_zoom"
  rg = (1747,2020) # range x axis

  p = figure(title= title,  x_range=rg, y_range=list(months), 
             x_axis_location="below", plot_width=900, plot_height=600,
             y_axis_label = "Month",x_axis_label = " ",
             tools=TOOLS, toolbar_location='right')

  p.grid.grid_line_color = None
  p.axis.axis_line_color = None
  p.axis.major_tick_line_color = None
  p.axis.major_label_text_font_size = "8.0pt"
  p.axis.axis_label_text_font_size = "9.0pt"
  p.axis.major_label_standoff = 0
  p.border_fill_color = "#e6e6e6"
  p.background_fill_color = "darkslategray"
  p.rect(x="Year", y="Month", width=0.99, height=1,
         source=source,
       fill_color={'field': 'Spots', 'transform': mapper},
       line_color=None)

  p.select_one(HoverTool).tooltips = [
       ('Date', '@Month @Year'),
       ('Spots', '@Spots{0.0}'),
  ]

  
  return p



colors = ["#ffffcc","#ffeda0","#fed976","#feb24c","#fd8d3c","#fc4e2a","#e31a1c","#bd0026","#800026"]
title = "Monthly Mean Total Sunspot Number -  Source : SIDC - Solar Influences Data Analysis Center."
heat = heatmap(mat_sun, colors, title, year1, m[1:])    
tab1 = Panel(child=heat, title="Heatmap")

# Scatter plot
def scatter(data, colors, title, year1) : 

  mapper = LinearColorMapper(palette=colors, low=data.Spots.min(), high=data.Spots.max())
  source = ColumnDataSource(data)
   

  TOOLS = "hover, pan,box_zoom,reset,wheel_zoom"
  rg = (-7.1e12,1.6e12) # Try to use 
  # rg = (pd.Timestamp('1744-01-31 00:00:00'),pd.Timestamp('2022-01-31 00:00:00'))
  # but only got error, so I adjusted by hand.
  
    
  p = figure(title= title, y_range=(-10,410), x_range=rg,
             x_axis_location="below", plot_width=900, plot_height=600,
             y_axis_label = "Sunspots",x_axis_label = " ", x_axis_type = "datetime",
             tools=TOOLS, toolbar_location='right')

  p.grid.grid_line_color = "gainsboro"
  p.axis.axis_line_color = "gainsboro"
  p.axis.major_tick_line_color = None
  p.axis.major_label_text_font_size = "8.0pt"
  p.axis.axis_label_text_font_size = "9.0pt"
  p.axis.major_label_standoff = 0
  #p.xaxis.major_label_orientation = np.pi / 3
  p.border_fill_color = "#e6e6e6"
  p.background_fill_color =  "white"
  
  p.circle(x="Date", y="Spots", size = 3,
       source=source,
       fill_color={'field': 'Spots', 'transform': mapper},
       line_color={'field': 'Spots', 'transform': mapper},
       fill_alpha=0.4, line_alpha = 1.0)

  p.select_one(HoverTool).tooltips = [
       ('Date', '@Data'),
       ('Spots', '@Spots{0.0}'),
       ('MA','@MA{0.0}')
  ]

  p.line(x="Date", y="MA",    source=source, legend = "33 month moving average ", line_width = 2.0, line_alpha = 0.90, line_color = "steelblue")
  p.line(x="Date", y="Spots", source=source, line_width = 0.2, line_alpha = 1.0, line_color = "coral")
  
  p.legend.location = "top_right"
  p.legend.label_text_font_size = "9pt"
  p.legend.spacing = 2
  p.legend.padding = 2
  p.legend.label_standoff = 2
  p.legend.margin = 10
  p.legend.background_fill_alpha = 1.0

  return p#tab1


colors = ["#ff8c00","#ff4500","#b22222","#8b0000"]
year1 = mat_sun["Date"]

scat = scatter(mat_sun, colors, title, year1)    

tab2 = Panel(child=scat, title="Monthly mean")

tabs = Tabs(tabs=[tab1,tab2])
show(tabs)




